## 0. Simulate some data and fit emulator

In [1]:
import torch

from autoemulate.experimental.simulations.projectile import ProjectileMultioutput
from autoemulate.experimental.emulators.gaussian_process.exact import (
    GaussianProcessExact,
)

In [2]:
sim = ProjectileMultioutput()
x = sim.sample_inputs(10)
y = sim.forward_batch(x)

Running simulations:   0%|          | 0/10 [00:00<?, ?it/s]

Running simulations: 100%|██████████| 10/10 [00:00<00:00, 659.90it/s]

Successfully completed 10/10 simulations (100.0%)


In [3]:
# check range of training outputs
torch.min(y, dim=0).values, torch.max(y, dim=0).values

(tensor([0.9270, 1.3262], dtype=torch.float64),
 tensor([29740.5185,   462.3643], dtype=torch.float64))

In [4]:
gp = GaussianProcessExact(x, y)
gp.fit(x, y)

## 1. Simple HMC example.

In [5]:
from autoemulate.experimental.calibration.hmc import HMCCalibrator

Lets start with an observation inside the training range, we should be able to recover the input parameters.

In [6]:
# given the ranges, we can actually just them to the same thing
observations = {name: 100 for name in sim.output_names}

In [7]:
hmc = HMCCalibrator(gp, sim.parameters_range, observations, 1.0)

In [ ]:
samples = hmc.run_mcmc(warmup_steps=10, num_samples=100)

In [12]:
samples['c'].mean(), samples['v0'].mean()

(tensor(-1.4926), tensor(667.2050))

## 2. Combining this with sensitivity analysis and history matching.